# Phase 3: Model Training

**Objective:** Train 6 ML models on all datasets and implement baseline methods

## Timeline: Week 4

### Tasks:
1. Train models on each dataset:
   - Logistic Regression / Linear Regression
   - Random Forest
   - XGBoost
   - Neural Network (MLP)
   - SVM
   - Decision Tree

2. Implement baseline comparison methods:
   - KernelSHAP
   - TreeExplainer (for tree-based models)

3. Save trained models for experiments

### Success Criteria:
- All models achieve reasonable accuracy
- Models saved to `data/models/`
- Baselines ready for Phase 4 comparison

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ML models
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score

# XGBoost
import xgboost as xgb

# SHAP baselines
import shap

# Setup paths
project_root = Path.cwd() / 'OPS_Project'
data_dir = project_root / 'data' / 'processed'
models_dir = project_root / 'data' / 'models'
models_dir.mkdir(exist_ok=True, parents=True)

print("✅ Libraries imported successfully!")
print(f"Working directory: {project_root}")
print(f"Models will be saved to: {models_dir}")

✅ Libraries imported successfully!
Working directory: c:\Users\Yash\Music\jisads research\OPS_Project
Models will be saved to: c:\Users\Yash\Music\jisads research\OPS_Project\data\models


## 1. Load All Datasets

In [8]:
# Load all 6 datasets
datasets = {}

dataset_names = [
    'iris',
    'california_housing',
    'adult_income',
    'mnist_pca',
    'synthetic_svm',
    'non_submodular'
]

for name in dataset_names:
    with open(data_dir / f'{name}.pkl', 'rb') as f:
        datasets[name] = pickle.load(f)
    print(f"✅ Loaded {name}: X shape {datasets[name]['X'].shape}, y shape {datasets[name]['y'].shape}")

print(f"\nTotal datasets loaded: {len(datasets)}")

✅ Loaded iris: X shape (150, 4), y shape (150,)
✅ Loaded california_housing: X shape (20640, 8), y shape (20640,)
✅ Loaded adult_income: X shape (5000, 14), y shape (5000,)
✅ Loaded mnist_pca: X shape (2000, 50), y shape (2000,)
✅ Loaded synthetic_svm: X shape (1000, 100), y shape (1000,)
✅ Loaded non_submodular: X shape (500, 10), y shape (500,)

Total datasets loaded: 6


## 2. Define Model Training Pipeline

In [12]:
def get_models(task_type='classification'):
    """
    Get 6 ML models for classification or regression.
    
    Args:
        task_type: 'classification' or 'regression'
        
    Returns:
        dict: Model name -> model instance
    """
    if task_type == 'classification':
        return {
            'logistic': LogisticRegression(max_iter=1000, random_state=42),
            'random_forest': RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10),
            'xgboost': xgb.XGBClassifier(n_estimators=100, random_state=42, max_depth=6, eval_metric='logloss'),
            'neural_net': MLPClassifier(hidden_layers=(100, 50), max_iter=500, random_state=42),
            'svm': SVC(kernel='rbf', probability=True, random_state=42),
            'decision_tree': DecisionTreeClassifier(max_depth=10, random_state=42)
        }
    else:  # regression
        return {
            'linear': LinearRegression(),
            'random_forest': RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10),
            'xgboost': xgb.XGBRegressor(n_estimators=100, random_state=42, max_depth=6),
            'neural_net': MLPRegressor(hidden_layers=(100, 50), max_iter=500, random_state=42),
            'svm': SVR(kernel='rbf'),
            'decision_tree': DecisionTreeRegressor(max_depth=10, random_state=42)
        }

def train_and_evaluate(X, y, model, task_type='classification'):
    """
    Train model and evaluate performance.
    
    Returns:
        model: Trained model
        metrics: Performance metrics
        X_train, X_test, y_train, y_test: Train/test splits
    """
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y if task_type == 'classification' else None
    )
    
    # Train model
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    
    if task_type == 'classification':
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred, average='weighted')
        }
    else:
        metrics = {
            'mse': mean_squared_error(y_test, y_pred),
            'r2': r2_score(y_test, y_pred)
        }
    
    return model, metrics, X_train, X_test, y_train, y_test

print("✅ Training pipeline defined")

✅ Training pipeline defined


## 3. Train Models on All Datasets

In [13]:
# Configuration: dataset -> task type
dataset_config = {
    'iris': 'classification',
    'california_housing': 'regression',
    'adult_income': 'classification',
    'mnist_pca': 'classification',
    'synthetic_svm': 'classification',
    'non_submodular': 'regression'
}

# Store all trained models and results
trained_models = {}
results_summary = []

print("=" * 80)
print("TRAINING MODELS ON ALL DATASETS")
print("=" * 80)

for dataset_name in dataset_names:
    print(f"\n{'='*80}")
    print(f"Dataset: {dataset_name.upper()}")
    print(f"{'='*80}")
    
    X = datasets[dataset_name]['X']
    y = datasets[dataset_name]['y']
    task_type = dataset_config[dataset_name]
    
    print(f"Task type: {task_type}")
    print(f"Data shape: X={X.shape}, y={y.shape}")
    
    # Get models for this task
    models = get_models(task_type)
    
    trained_models[dataset_name] = {}
    
    for model_name, model in models.items():
        print(f"\n  Training {model_name}...", end=' ')
        
        try:
            trained_model, metrics, X_train, X_test, y_train, y_test = train_and_evaluate(
                X, y, model, task_type
            )
            
            # Store model and splits
            trained_models[dataset_name][model_name] = {
                'model': trained_model,
                'X_train': X_train,
                'X_test': X_test,
                'y_train': y_train,
                'y_test': y_test,
                'metrics': metrics,
                'task_type': task_type
            }
            
            # Save model
            model_path = models_dir / f'{dataset_name}_{model_name}.pkl'
            with open(model_path, 'wb') as f:
                pickle.dump(trained_models[dataset_name][model_name], f)
            
            # Print metrics
            if task_type == 'classification':
                print(f"Acc: {metrics['accuracy']:.4f}, F1: {metrics['f1_score']:.4f} ✅")
            else:
                print(f"MSE: {metrics['mse']:.4f}, R²: {metrics['r2']:.4f} ✅")
            
            # Record summary
            results_summary.append({
                'dataset': dataset_name,
                'model': model_name,
                'task': task_type,
                **metrics
            })
            
        except Exception as e:
            print(f"FAILED: {e} ❌")

print("\n" + "=" * 80)
print("✅ MODEL TRAINING COMPLETE")
print("=" * 80)
print(f"Total models trained: {len(results_summary)}")
print(f"Models saved to: {models_dir}")

TRAINING MODELS ON ALL DATASETS

Dataset: IRIS
Task type: classification
Data shape: X=(150, 4), y=(150,)


TypeError: MLPClassifier.__init__() got an unexpected keyword argument 'hidden_layers'

## 4. Results Summary Table

In [6]:
# Create summary DataFrame
results_df = pd.DataFrame(results_summary)

print("\n" + "=" * 80)
print("MODEL PERFORMANCE SUMMARY")
print("=" * 80)

# Classification results
print("\nCLASSIFICATION TASKS:")
print("-" * 80)
clf_results = results_df[results_df['task'] == 'classification']
if len(clf_results) > 0:
    print(clf_results[['dataset', 'model', 'accuracy', 'f1_score']].to_string(index=False))

# Regression results
print("\n\nREGRESSION TASKS:")
print("-" * 80)
reg_results = results_df[results_df['task'] == 'regression']
if len(reg_results) > 0:
    print(reg_results[['dataset', 'model', 'mse', 'r2']].to_string(index=False))

# Save summary
results_df.to_csv(project_root / 'results' / 'model_training_summary.csv', index=False)
print(f"\n✅ Summary saved to: {project_root / 'results' / 'model_training_summary.csv'}")


MODEL PERFORMANCE SUMMARY

CLASSIFICATION TASKS:
--------------------------------------------------------------------------------


KeyError: 'task'

## 5. Implement Baseline Methods (KernelSHAP & TreeExplainer)

In [7]:
class BaselineMethods:
    """
    Wrapper for baseline Shapley estimation methods.
    Implements KernelSHAP and TreeExplainer for comparison.
    """
    
    @staticmethod
    def kernelshap(model, X_background, X_explain, n_samples=1000):
        """
        Estimate Shapley values using KernelSHAP.
        
        Args:
            model: Trained model
            X_background: Background dataset for sampling
            X_explain: Data points to explain
            n_samples: Number of samples for kernel approximation
            
        Returns:
            shap_values: SHAP values (n_samples, n_features)
        """
        # Create KernelExplainer
        if hasattr(model, 'predict_proba'):
            explainer = shap.KernelExplainer(model.predict_proba, X_background)
        else:
            explainer = shap.KernelExplainer(model.predict, X_background)
        
        # Compute SHAP values
        shap_values = explainer.shap_values(X_explain, nsamples=n_samples)
        
        return shap_values
    
    @staticmethod
    def tree_explainer(model, X_explain):
        """
        Estimate Shapley values using TreeExplainer (for tree-based models).
        
        Args:
            model: Trained tree-based model
            X_explain: Data points to explain
            
        Returns:
            shap_values: Exact SHAP values (n_samples, n_features)
        """
        # Create TreeExplainer
        explainer = shap.TreeExplainer(model)
        
        # Compute SHAP values
        shap_values = explainer.shap_values(X_explain)
        
        return shap_values
    
    @staticmethod
    def is_tree_based(model):
        """
        Check if model is tree-based (can use TreeExplainer).
        """
        tree_models = (
            RandomForestClassifier, RandomForestRegressor,
            DecisionTreeClassifier, DecisionTreeRegressor,
            xgb.XGBClassifier, xgb.XGBRegressor
        )
        return isinstance(model, tree_models)

print("✅ Baseline methods (KernelSHAP, TreeExplainer) implemented")

✅ Baseline methods (KernelSHAP, TreeExplainer) implemented


## 6. Test Baseline Methods

In [ ]:
# Quick test on Iris dataset with Random Forest
print("Testing baseline methods on Iris dataset (Random Forest)...\n")

test_dataset = 'iris'
test_model_name = 'random_forest'

model_data = trained_models[test_dataset][test_model_name]
model = model_data['model']
X_test = model_data['X_test'][:5]  # Test on 5 samples
X_background = model_data['X_train'][:100]  # Background for KernelSHAP

print(f"Model: {test_model_name}")
print(f"Test samples: {X_test.shape}")
print(f"Background samples: {X_background.shape}")

# TreeExplainer (exact for tree models)
if BaselineMethods.is_tree_based(model):
    print("\n1. TreeExplainer (exact):")
    import time
    t0 = time.time()
    shap_tree = BaselineMethods.tree_explainer(model, X_test)
    t_tree = time.time() - t0
    
    if isinstance(shap_tree, list):  # Multi-class
        shap_tree = shap_tree[0]  # Use first class
    
    print(f"   Shape: {shap_tree.shape}")
    print(f"   Time: {t_tree*1000:.2f} ms")
    print(f"   Sample values (1st instance): {shap_tree[0]}")

# KernelSHAP (approximation)
print("\n2. KernelSHAP (n_samples=100):")
t0 = time.time()
shap_kernel = BaselineMethods.kernelshap(model, X_background, X_test, n_samples=100)
t_kernel = time.time() - t0

if isinstance(shap_kernel, list):  # Multi-class
    shap_kernel = shap_kernel[0]

print(f"   Shape: {shap_kernel.shape}")
print(f"   Time: {t_kernel*1000:.2f} ms")
print(f"   Sample values (1st instance): {shap_kernel[0]}")

print("\n✅ Baseline methods tested successfully!")

## 7. Phase 3 Summary

In [ ]:
print("=" * 80)
print("PHASE 3 COMPLETE: MODEL TRAINING")
print("=" * 80)

print("\n✅ Achievements:")
print(f"   - Trained {len(results_summary)} models across 6 datasets")
print(f"   - 6 model types: Logistic/Linear, Random Forest, XGBoost, Neural Net, SVM, Decision Tree")
print(f"   - All models saved to: {models_dir}")
print(f"   - Baseline methods implemented: KernelSHAP, TreeExplainer")
print(f"   - Performance summary saved: {project_root / 'results' / 'model_training_summary.csv'}")

print("\n📊 Model Performance:")
print(f"   - Classification tasks: {len(clf_results)} models")
print(f"     Average accuracy: {clf_results['accuracy'].mean():.4f}")
print(f"   - Regression tasks: {len(reg_results)} models")
print(f"     Average R²: {reg_results['r2'].mean():.4f}")

print("\n🎯 Next: Phase 4 - Experimental Evaluation")
print("   Run full experiments: 6 datasets × 6 models × 5 algorithms × 5 budgets")
print("   Total experiments: 900+ configurations")

print("\n" + "=" * 80)